In [8]:
import numpy as np
import pandas as pd


In [9]:
pred_dt = pd.read_csv("sub-dt-20170502-184700.csv")
pred_elasticnet = pd.read_csv("sub-elasticnet-20170502-184916.csv")
pred_et = pd.read_csv("sub-et-20170502-184844.csv")
pred_lars = pd.read_csv("sub-lars-20170502-184918.csv")
pred_lasso = pd.read_csv("sub-lasso-20170502-184913.csv")
pred_lgb = pd.read_csv("sub-lgb-20170502-184911.csv")
pred_rf = pd.read_csv("sub-rf-20170502-184840.csv")
pred_ridge = pd.read_csv("sub-ridge-20170502-184914.csv")
pred_svm = pd.read_csv("sub-svm-rbf-20170502-184923.csv")
pred_xgb = pd.read_csv("sub-xgb-20170502-184907.csv")